<a href="https://colab.research.google.com/github/blockmoo/Python/blob/main/%EC%84%A4%EB%AC%B8%EC%A1%B0%EC%82%AC_%EC%A3%BC%EA%B4%80%EC%8B%9D_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.chdir("/content/drive/MyDrive/설문지 분석/주관식")

In [ ]:
!pip install konlpy wordcloud

In [ ]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from collections import Counter
import sys
from wordcloud import WordCloud
from konlpy.tag import Twitter
import re
import nltk
import string # 단어 찾기

In [ ]:
dt19 = pd.read_csv("2019_text.csv", encoding="cp949")
dt19.head()

,연도,구분 ID,진단Level,평가자그룹명,42. 진단대상자의 '강점' 을 관찰한 행동 및 사례 중심으로 구체적으로 기술해 주십시오.\n(※ 본인일 경우 본인의 강점 기술),43. 진단대상자의 보완할 점이나 개발이 필요한 부분을 관찰한 행동 및 사례 중심으로 구체적으로 기술해 주십시오.\n(※ 본인일 경우 본인의 개발 필요점 기술),"39. 강점/보완점 외에, 진단대상자의 리더십 역량 향상에 도움이 될만한 제언/의견을 자유롭게 기술해 주십시오."
0,2019,ki0341,L1,본인,업무든 개인역량이든 솔선수범하여 업무 진행하는 스타일,미완된 과제에 대해서 끝까지 챙기는 부분 부족,본인이 실무적인 일을 하기 보다는 주로 부하직원들이 막혀 있는 요소가 무엇인지 찾고...
1,2019,dh0575,L1,본인,맡은바 업무를 완수하기 위해 최선의 노력을 다함.,부서원들에게 성과에 대해 주기적인 Feedback(1:1 미팅)이 부족함.,특이사항 없음.
2,2019,cy0680,L1,본인,책임감이 강하다,사업부 협업을 더욱 강하게 이끌 필요가 있다.,사업부 경험
3,2019,hy0227,L2,본인,"분석력. 예측력, 추진력",다혈질적 성향,.
4,2019,kw7154,L1,본인,후배 사원들을 이해하고 소통하려고 노력함.,때로 다소 급한성격에 빠른 결과를 바라는 경향이 있음.,개개인의 성향을 잘알고 각자에 맞는 제언이나 도움을 주려고 노력중임.


In [ ]:
본인19 = dt19["평가자그룹명"] == "본인"
상사19 = dt19["평가자그룹명"] == "상사"
부서원19 = dt19["평가자그룹명"] == "부서원"
동료19 = dt19["평가자그룹명"] == "동료"
타인19 = dt19["평가자그룹명"] != "본인"
dt19[타인19].tail(10)

,연도,구분 ID,진단Level,평가자그룹명,42. 진단대상자의 '강점' 을 관찰한 행동 및 사례 중심으로 구체적으로 기술해 주십시오.\n(※ 본인일 경우 본인의 강점 기술),43. 진단대상자의 보완할 점이나 개발이 필요한 부분을 관찰한 행동 및 사례 중심으로 구체적으로 기술해 주십시오.\n(※ 본인일 경우 본인의 개발 필요점 기술),"39. 강점/보완점 외에, 진단대상자의 리더십 역량 향상에 도움이 될만한 제언/의견을 자유롭게 기술해 주십시오."
37544,2019,sm2020,L1,부서원,"유연한 사고 및 후배 업무의 결정권 부여, 제한 없는 의사 소통",지금보다 더 지속할 수 있는 건강관리 필요,.
37545,2019,pa1940,L1,부서원,.,.,.
37546,2019,pa1940,L1,부서원,여러사람의 의견을 청취하고 최선의 결정을 하려고 행동합니다.,도전적인 부서업무 목표를 수립하여 추진하면 좋겠습니다.,의견을 듣고 논의하여 결정하는 업무진행이 좋습니다.
37547,2019,pa1940,L1,부서원,설비 5D 및 SEUP 업무 관련 유연한 사고 방식으로 담당자에게 매사 적극적인 도...,"항상 부서원들을 관심 가져 주셔서, 항상 고마운 마음으로 만족하고 있습니다.",항상 도움주셔서 감사합니다.
37548,2019,sy0796,L1,부서원,목표 달성을 위해 우리가 어떻게 해나가야 하는지 알려주는 상사,잘모르겠습니다.,부서원의 부족한 점에 대해 좀더 말을 해주면 좋을 것 같습니다.
37549,2019,hd0212,L3,부서원,"본인의 책임의식, 신속한 의사 결정으로 부서원들의 고충을 해결해주며 후배 사원 육성...",현장의 소리를 빨리 직접 듣는것은 매우 바람직하고 현장에서 많은 동기 부여를 받고 ...,노동적인 제조의 고질적 문제를 시스템적으로 변화하는 제조가 되게 도와 주십시요.
37550,2019,zi0870,L1,동료,-,-,-
37551,2019,dw2021,L2,부서원,업무 목표에 대한 계획 및 추진력,수고하세요,수고하세요
37552,2019,ik1074,L2,상사,책임감있고 리더십을 가지고 조직을 잘 이끌어 나간다.,어학 능력을 더 키워서 글로벌한 업무를 잘 할 수 있도록 한다.,보다 도전적인 과제를 수행하면서 성공의 경험을 더 많이 쌓는다.
37553,2019,kw1075,L2,상사,"지식 습득 능력, 소통 능력이 뛰어납니다.",추진력 보완을 하면 좋을 것 같습니다.,"지금 같이 하시되, 좀더 자신감을 갖고 하시면 좋을 것 같습니다."


In [ ]:
dt20 = pd.read_csv("2020_text.csv", encoding="cp949")
dt20.head(3)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,연도,구분ID,진단Level,직급,직책,임원여부,대직무,평가자그룹명,42. 진단대상자의 '강점' 을 관찰한 행동 및 사례 중심으로 구체적으로 기술해 주십시오.\n(※ 본인일 경우 본인의 강점 기술),43. 진단대상자의 보완할 점이나 개발이 필요한 부분을 관찰한 행동 및 사례 중심으로 구체적으로 기술해 주십시오.\n(※ 본인일 경우 본인의 개발 필요점 기술)
0,2020.0,le1195,L1,CL4,NaN,NaN,인사 (JFG),본인,부서원과 주 2~3회 소통하고 있으며 업무상의 자율성을 부여하여 본인의 업무에 ow...,때때로 문제의 본질보다는 현상에 주목하여 문제해결에 시간을 낭비하는 경우가 있음.\...
1,2020.0,ke1195,L1,CL4,NaN,NaN,영업 (JFG),본인,다양한 정보와 분석을 통해 의사결정 및 문제해결에 적합한 솔루션을 제시함. 동료 들...,"파트원들의 업무 역량 향상 및 동기부여에 좀 더 노력이 필요하며, 전체를 보는 통찰..."
2,2020.0,so1196,L1,CL4,NaN,NaN,연구개발 (JFG),본인,DRAM 설계라는 한 분야에서 20년이 넘는 시간동안 다양한 제품개발을 통해 많은...,타 부서와의 지속적인 협업을 통해 부서 업무를 개선할 수 있는 방안을 모색해 나가야...


In [ ]:
본인20 = dt20["평가자그룹명"] == "본인"
상사20 = dt20["평가자그룹명"] == "상사"
부서원20 = dt20["평가자그룹명"] == "부서원"
동료20 = dt20["평가자그룹명"] == "동료"
타인20 = dt20["평가자그룹명"] != "본인"
dt20[타인20].head(3)

,연도,구분ID,진단Level,직급,직책,임원여부,대직무,평가자그룹명,42. 진단대상자의 '강점' 을 관찰한 행동 및 사례 중심으로 구체적으로 기술해 주십시오.\n(※ 본인일 경우 본인의 강점 기술),43. 진단대상자의 보완할 점이나 개발이 필요한 부분을 관찰한 행동 및 사례 중심으로 구체적으로 기술해 주십시오.\n(※ 본인일 경우 본인의 개발 필요점 기술)
1530,2020.0,sh1239,L1,CL4,NaN,NaN,연구개발 (JFG),상사,난이도 높은 업무를 끝까지 책임지고 해결하려는 노력과 자세가 우수함,중국어등 어학향상에 노력이 필요함
1531,2020.0,sh1444,L2,CL4,PL,NaN,연구개발 (JFG),상사,강한 책임감과 뛰어난 업무 추진력. 상사의 지시에 복종하는 충성심.,아랫 사람에게 강압적으로 보일 수 있는 언행이 간혹 보여 이 점을 보완할 필요가 있음.
1532,2020.0,ts1488,L1,CL4,NaN,NaN,연구개발 (JFG),상사,기술적 data에 대한 검증을 진핸한다.,계획과 일정을 지속적으로 살핀다.


In [ ]:
dt = pd.read_excel("../Raw data/2021년.xlsx")
dt21 = dt.iloc[:,[0,1,2,3,4,5,32,33]]
dt21.head(3)

,진단년도,구분ID,진단Level,직급,임원여부,평가자그룹명,29. 진단대상자의 '강점' 을 관찰한 행동 및 사례 중심으로 구체적으로 기술해 주십시오.\r\r\r\n(※ 본인일 경우 본인의 강점 기술),30. 진단대상자의 보완할 점이나 개발이 필요한 부분을 관찰한 행동 및 사례 중심으로 구체적으로 기술해 주십시오.\r\r\r\n(※ 본인일 경우 본인의 개발 필요점 기술)
0,2021,a71250,L2,CL4,NaN,본인,수명업무에 대해 합리적이고 효율적인 해결방안을 도출하고 실행하는 업무 처리방식,업무의 확장을 위한 공정/설비기술 전반적인 학습이 필요함
1,2021,a71250,L2,CL4,NaN,상사,"꾸준하며, 본인 업의 본질을 잘 파악하고 최선을 다한다",욕심을 좀 더 가지고 전문 인력 양성에 좀 더 힘쓰면 좋을것 같음
2,2021,a71250,L2,CL4,NaN,부서원,개개인의 면담을 통하여 업무 전문성을 육성하고 자유롭게 업무를 진행 할 수 있게 함,...


In [ ]:
본인21 = dt21["평가자그룹명"] == "본인"
상사21 = dt21["평가자그룹명"] == "상사"
부서원21 = dt21["평가자그룹명"] == "부서원"
동료21 = dt21["평가자그룹명"] == "동료"
타인21 = dt21["평가자그룹명"] != "본인"
dt21[타인21].head(3)

,진단년도,구분ID,진단Level,직급,임원여부,평가자그룹명,29. 진단대상자의 '강점' 을 관찰한 행동 및 사례 중심으로 구체적으로 기술해 주십시오.\r\r\r\n(※ 본인일 경우 본인의 강점 기술),30. 진단대상자의 보완할 점이나 개발이 필요한 부분을 관찰한 행동 및 사례 중심으로 구체적으로 기술해 주십시오.\r\r\r\n(※ 본인일 경우 본인의 개발 필요점 기술)
1,2021,a71250,L2,CL4,NaN,상사,"꾸준하며, 본인 업의 본질을 잘 파악하고 최선을 다한다",욕심을 좀 더 가지고 전문 인력 양성에 좀 더 힘쓰면 좋을것 같음
2,2021,a71250,L2,CL4,NaN,부서원,개개인의 면담을 통하여 업무 전문성을 육성하고 자유롭게 업무를 진행 할 수 있게 함,...
3,2021,a71250,L2,CL4,NaN,부서원,중장기적인 개선항목 발굴후 적절하게 인원분배가 이루어짐,.


임원과 비임원 비교

In [ ]:
dat19 = pd.read_csv("2019_공유용.csv", encoding="cp949", header=1)

In [ ]:
data19 = dat19.iloc[:,[0,1,2,3,4,5,6,50,51]]
data19.head(3)

,연도,구분 ID,진단Level,직책,임원여부,대직무,평가자그룹명,42. 진단대상자의 '강점' 을 관찰한 행동 및 사례 중심으로 구체적으로 기술해 주십시오.\n(※ 본인일 경우 본인의 강점 기술),43. 진단대상자의 보완할 점이나 개발이 필요한 부분을 관찰한 행동 및 사례 중심으로 구체적으로 기술해 주십시오.\n(※ 본인일 경우 본인의 개발 필요점 기술)
0,2019,ki0341,L1,NaN,NaN,인사 (JFG),본인,업무든 개인역량이든 솔선수범하여 업무 진행하는 스타일,미완된 과제에 대해서 끝까지 챙기는 부분 부족
1,2019,dh0575,L1,NaN,NaN,인사 (JFG),본인,맡은바 업무를 완수하기 위해 최선의 노력을 다함.,부서원들에게 성과에 대해 주기적인 Feedback(1:1 미팅)이 부족함.
2,2019,cy0680,L1,NaN,NaN,인사 (JFG),본인,책임감이 강하다,사업부 협업을 더욱 강하게 이끌 필요가 있다.


In [ ]:
dat20 = pd.read_csv("2020_공유용.csv", encoding="cp949", header=1)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2,3,4,5,6,7,60,61) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
data20 = dat20.iloc[:,[0,1,2,3,4,5,6,7,60,61]]
data20.head(3)

,연도,구분ID,진단Level,직급,직책,임원여부,대직무,평가자그룹명,42. 진단대상자의 '강점' 을 관찰한 행동 및 사례 중심으로 구체적으로 기술해 주십시오.\n(※ 본인일 경우 본인의 강점 기술),43. 진단대상자의 보완할 점이나 개발이 필요한 부분을 관찰한 행동 및 사례 중심으로 구체적으로 기술해 주십시오.\n(※ 본인일 경우 본인의 개발 필요점 기술)
0,2020.0,le1195,L1,CL4,NaN,NaN,인사 (JFG),본인,부서원과 주 2~3회 소통하고 있으며 업무상의 자율성을 부여하여 본인의 업무에 ow...,때때로 문제의 본질보다는 현상에 주목하여 문제해결에 시간을 낭비하는 경우가 있음.\...
1,2020.0,ke1195,L1,CL4,NaN,NaN,영업 (JFG),본인,다양한 정보와 분석을 통해 의사결정 및 문제해결에 적합한 솔루션을 제시함. 동료 들...,"파트원들의 업무 역량 향상 및 동기부여에 좀 더 노력이 필요하며, 전체를 보는 통찰..."
2,2020.0,so1196,L1,CL4,NaN,NaN,연구개발 (JFG),본인,DRAM 설계라는 한 분야에서 20년이 넘는 시간동안 다양한 제품개발을 통해 많은...,타 부서와의 지속적인 협업을 통해 부서 업무를 개선할 수 있는 방안을 모색해 나가야...


In [ ]:
임원본인19 = (data19.임원여부 == "임원/FM") & (data19.평가자그룹명 == "본인")
임원타인19 = (data19.임원여부 == "임원/FM") & (data19.평가자그룹명 != "본인")
비임원본인19 = (data19.임원여부 != "임원/FM") & (data19.평가자그룹명 == "본인")
비임원타인19 = (data19.임원여부 != "임원/FM") & (data19.평가자그룹명 != "본인")
data19[비임원본인19].head(3)

,연도,구분 ID,진단Level,직책,임원여부,대직무,평가자그룹명,42. 진단대상자의 '강점' 을 관찰한 행동 및 사례 중심으로 구체적으로 기술해 주십시오.\n(※ 본인일 경우 본인의 강점 기술),43. 진단대상자의 보완할 점이나 개발이 필요한 부분을 관찰한 행동 및 사례 중심으로 구체적으로 기술해 주십시오.\n(※ 본인일 경우 본인의 개발 필요점 기술)
0,2019,ki0341,L1,NaN,NaN,인사 (JFG),본인,업무든 개인역량이든 솔선수범하여 업무 진행하는 스타일,미완된 과제에 대해서 끝까지 챙기는 부분 부족
1,2019,dh0575,L1,NaN,NaN,인사 (JFG),본인,맡은바 업무를 완수하기 위해 최선의 노력을 다함.,부서원들에게 성과에 대해 주기적인 Feedback(1:1 미팅)이 부족함.
2,2019,cy0680,L1,NaN,NaN,인사 (JFG),본인,책임감이 강하다,사업부 협업을 더욱 강하게 이끌 필요가 있다.


In [ ]:
임원본인20 = (data20.임원여부 == "임원/FM") & (data20.평가자그룹명 == "본인")
임원타인20 = (data20.임원여부 == "임원/FM") & (data20.평가자그룹명 != "본인")
비임원본인20 = (data20.임원여부 != "임원/FM") & (data20.평가자그룹명 == "본인")
비임원타인20 = (data20.임원여부 != "임원/FM") & (data20.평가자그룹명 != "본인")
data20[비임원본인20].head(3)

,연도,구분ID,진단Level,직급,직책,임원여부,대직무,평가자그룹명,42. 진단대상자의 '강점' 을 관찰한 행동 및 사례 중심으로 구체적으로 기술해 주십시오.\n(※ 본인일 경우 본인의 강점 기술),43. 진단대상자의 보완할 점이나 개발이 필요한 부분을 관찰한 행동 및 사례 중심으로 구체적으로 기술해 주십시오.\n(※ 본인일 경우 본인의 개발 필요점 기술)
0,2020.0,le1195,L1,CL4,NaN,NaN,인사 (JFG),본인,부서원과 주 2~3회 소통하고 있으며 업무상의 자율성을 부여하여 본인의 업무에 ow...,때때로 문제의 본질보다는 현상에 주목하여 문제해결에 시간을 낭비하는 경우가 있음.\...
1,2020.0,ke1195,L1,CL4,NaN,NaN,영업 (JFG),본인,다양한 정보와 분석을 통해 의사결정 및 문제해결에 적합한 솔루션을 제시함. 동료 들...,"파트원들의 업무 역량 향상 및 동기부여에 좀 더 노력이 필요하며, 전체를 보는 통찰..."
2,2020.0,so1196,L1,CL4,NaN,NaN,연구개발 (JFG),본인,DRAM 설계라는 한 분야에서 20년이 넘는 시간동안 다양한 제품개발을 통해 많은...,타 부서와의 지속적인 협업을 통해 부서 업무를 개선할 수 있는 방안을 모색해 나가야...


In [ ]:
임원본인21 = (dt21.임원여부 == "임원") & (dt21.평가자그룹명 == "본인")
임원타인21 = (dt21.임원여부 == "임원") & (dt21.평가자그룹명 != "본인")
비임원본인21 = (dt21.임원여부 != "임원") & (dt21.평가자그룹명 == "본인")
비임원타인21 = (dt21.임원여부 != "임원") & (dt21.평가자그룹명 != "본인")
dt21[비임원본인21].head(3)

,진단년도,구분ID,진단Level,직급,임원여부,평가자그룹명,29. 진단대상자의 '강점' 을 관찰한 행동 및 사례 중심으로 구체적으로 기술해 주십시오.\r\r\r\n(※ 본인일 경우 본인의 강점 기술),30. 진단대상자의 보완할 점이나 개발이 필요한 부분을 관찰한 행동 및 사례 중심으로 구체적으로 기술해 주십시오.\r\r\r\n(※ 본인일 경우 본인의 개발 필요점 기술)
0,2021,a71250,L2,CL4,NaN,본인,수명업무에 대해 합리적이고 효율적인 해결방안을 도출하고 실행하는 업무 처리방식,업무의 확장을 위한 공정/설비기술 전반적인 학습이 필요함
24,2021,aa1521,L1,CL3 이하,NaN,본인,문제의 핵심을 파악하고 신속한 의사 결정,부서원들 동기 부여하는 skill 향상이 필요함
44,2021,ab1292,L1,CL3 이하,NaN,본인,편안하게 다가가려고 노력하고 동료애를 느끼게 해주려고 하고있음,업무에 대한 지식부족과 더 좋은 환경의 교육을 만들어 줘야할 것 같음


2019년도 strength 명사만 추출

In [ ]:
### "없음"으로 답한 비율
dt21[비임원본인21].iloc[1,7].find("없")
dt21[비임원본인21].apply(lambda x : x.iloc[,7], axis = 0)

-1

## 트위터 단어 추출

In [ ]:
!pip install customized_konlpy

In [ ]:
def twitter_noun(temp, dic):
     #okt 객체 생성
    twitter = Twitter()
    # twitter.add_dictionary(dic, 'Noun')
    noun = twitter.nouns(temp)
    nltk.download('punkt')
    stopwords = dic
    clean_words = [] 
    for word in noun: 
        if word not in stopwords: #불용어 제거
            clean_words.append(word)  
    for i,v in enumerate(clean_words):
        if len(v)<2:
                clean_words.pop(i)

    count = Counter(clean_words)

    #명사 빈도 카운트
    noun_list = count.most_common(100)
    return noun_list

In [ ]:
list1 = []
# data = dt21[타인21].iloc[:,7]
data = dt21[비임원본인21].iloc[:,7]
for i in data:
  # item = i.replace("다양한","쩡보")
  # item = item.replace("에","?")
  # list1.append(item)
  list1.append(i)
list1 = [str(x) for x in list1 ]  # 숫자형이 있으면 문자형으로 변환 (이유 : 숫자형은 .join이 안됨.)
list2 = '\n'.join(list1)
print(list2[0:20])

업무의 확장을 위한 공정/설비기술 전


In [ ]:
# filename = "./데이터 중간단계/me_weak_2019.csv"
# f = open(filename, "r", encoding="utf-8")
# temp = f.read()
dic = ['업무','부서', '노력', '진행', '대한', '위해', '통해', '바탕', '대해', '가지', '관련', '통한', '기반', '부분', '수', '또한', '유관',
       '필요','보완','본인','생각','더','조금','특별','경우','서원','도록','파트','부족','극적','좀더','사항','모습','시간','해주','이나',
       '다른','요하','것','다소','현재','주시','원','로서','까지','족하','려고', "항상",'해주시',"가끔","상황","개발", "리더", "강화", "지금"]
noun_list = twitter_noun(list2, dic)

noun_list[0:10]

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[('기술', 563),
 ('역량', 546),
 ('능력', 504),
 ('소통', 355),
 ('관리', 291),
 ('향상', 271),
 ('지식', 265),
 ('조직', 221),
 ('추진', 218),
 ('전문', 201)]

In [ ]:
import csv
# csv 파일에 저장
# with open("./result/타인_21_단점.csv", "w", newline='', encoding="utf-8") as f:
with open("./result/비임원본인_21_단점.csv", "w", newline='', encoding="utf-8") as f:

  csvw = csv.writer(f)
  for v in noun_list:
    csvw.writerow(v)  

In [ ]:
# 워드 클라우드
def visualize(noun_list):

  wc = WordCloud(font_path = "./데이터 중간단계/H2GTRM.TTF", \
                 background_color = "white", \
                 width=1000, \
                 height=1000, \
                 max_words=100,\
                 max_font_size=300)
  wc.generate_from_frequencies(dict(noun_list))
  wc.to_file('./워드클라우드/keyword_other_weak_2019.png')


visualize(noun_list)

In [ ]:
### 단점이 "없다" 혹은 "."으로 기재한 비율
new = dt21[동료].iloc[:,7]
new.str.count("없")

18       0.0
19       0.0
20       1.0
21       0.0
22       0.0
        ... 
89348    1.0
89357    0.0
89358    0.0
89359    0.0
89360    1.0
Name: 30. 진단대상자의 보완할 점이나 개발이 필요한 부분을 관찰한 행동 및 사례 중심으로 구체적으로 기술해 주십시오.\r\r\r\n(※ 본인일 경우 본인의 개발 필요점 기술), Length: 20597, dtype: float64